In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nb_004c import *

# Dogs and cats

## Basic data aug

In [ ]:
PATH = Path('../../data/dogscats')

In [ ]:
data_mean, data_std = map(tensor, ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))
data_norm,data_denorm = normalize_funcs(data_mean,data_std)

In [ ]:
train_tfms = [flip_lr_tfm(p=0.5),
              rotate_tfm(degrees=(-10,10.), p=0.25),
              zoom_tfm(scale=(0.8,1.2), p=0.25),
              crop_pad_tfm(row_pct=(0,1.), col_pct=(0,1.))]

valid_tfms = [zoom_tfm(),
              crop_pad_tfm()]

In [ ]:
train_ds = FilesDataset.from_folder(PATH/'train')
valid_ds = FilesDataset.from_folder(PATH/'valid')

In [ ]:
sz = 224
train_tds = TfmDataset(train_ds, train_tfms, size=sz, do_crop=True)
valid_tds = TfmDataset(valid_ds, valid_tfms, size=sz, do_crop=True)

In [ ]:
data = DataBunch.create(train_tds, valid_tds, bs=64, num_workers=8, dl_tfms=data_norm)

In [ ]:
(x,y) = next(iter(data.train_dl))

In [ ]:
_,axs = plt.subplots(4,4,figsize=(12,12))
for i,ax in enumerate(axs.flatten()):
    img = data_denorm(x[i].cpu())
    show_image(img, ax)

## Model with a new head

In [ ]:
from torchvision.models import resnet34

In [ ]:
arch = resnet34

In [ ]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap, self.mp = nn.AdaptiveAvgPool2d(sz), nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [ ]:
def create_skeleton(model, cut):
    layers = list(model.children())
    if cut != 0: layers = layers[:-cut]
    layers = layers + [AdaptiveConcatPool2d(), Flatten()]
    return nn.Sequential(*layers)

In [ ]:
def num_features(m):
    c=list(m.children())
    if len(c)==0: return None
    for l in reversed(c):
        if hasattr(l, 'num_features'): return l.num_features
        res = num_features(l)
        if res is not None: return res

In [ ]:
model = create_skeleton(arch(), 2)

In [ ]:
num_features(model)

In [ ]:
def bn_dp_lin(n_in, n_out, bn=True, dp=0., actn=None):
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if dp != 0: layers.append(nn.Dropout(dp))
    layers.append(nn.Linear(n_in, n_out))
    if actn is not None: layers.append(actn)
    return layers

In [ ]:
def create_head(nf, nc, lin_ftrs=None, dps=None):
    lin_ftrs = [nf, 512, nc] if lin_ftrs is None else [nf] + lin_ftrs + [nc]
    if dps is None: dps = [0.25] * (len(lin_ftrs)-2) + [0.5]
    actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs)-2) + [None]
    layers = []
    for ni,no,dp,actn in zip(lin_ftrs[:-1],lin_ftrs[1:],dps,actns): 
        layers += bn_dp_lin(ni,no,True,dp,actn)
    return nn.Sequential(*layers)

In [ ]:
create_head(512, 2)

In [ ]:
class ConvLearner(Learner):
    
    def __init__(self, data, arch, cut, pretrained=True, lin_ftrs=None, dps=None, **kwargs):
        self.skeleton = create_skeleton(arch(pretrained), cut)
        nf = num_features(self.skeleton) * 2
        self.head = create_head(nf, len(data.train_ds.classes), lin_ftrs, dps)
        model = nn.Sequential(self.skeleton, self.head)
        super().__init__(data, model, **kwargs)
    
    def freeze(self):
        for p in self.skeleton.parameters(): p.require_grad = False
    
    def unfreeze(self):
        for p in self.skeleton.parameters(): p.require_grad = True

In [ ]:
learn = ConvLearner(data, arch, 2)

In [ ]:
learn.freeze()

In [ ]:
lr_find(learn)

In [ ]:
learn.recorder.plot()

In [ ]:
learn = ConvLearner(data, arch, 2)
learn.metrics = [accuracy]
learn.freeze()

In [ ]:
learn.fit(1, 4e-2)